In [22]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import itertools

In [30]:
base_url = "https://visitseattle.org/events/page/"
pages = 35
events_ls = []
for pg in range (1, pages+1):
    url = f'{base_url}{pg}'
    res = requests.get(url)

    soup = BeautifulSoup(res.text, "html.parser")
    selector = "div.search-result-preview > div > h3 > a"

    a_eles = soup.select(selector)

    events_ls.append([x['href'] for x in a_eles])

In [31]:
# Flatten the list
events_ls = list(itertools.chain.from_iterable(events_ls))

In [32]:
# Check if all pages have been paginated through
len(events_ls)

315

In [35]:
for event in events_ls:
    # Scrape the pages
    response = requests.get(event).text
    soup = BeautifulSoup(response, 'html.parser')

    # Extract the details of the event: name, date, location, type, region
    names = soup.find(class_="page-title").text
    date_location = soup.find('h4').text.split(" | ")
    type = soup.find(class_="button big medium black category").text
    region = soup.select_one('a.button.big.medium.black.category[href*=event_regions]').text

    # Look up location
    location_query = region

    if '/' in location_query:
        location_query = location_query.split(' / ')[0].strip()
    location_query = [f'{location_query}, Seattle']

    # Call API endpoint for location
    location_url = "https://nominatim.openstreetmap.org/search"
    params = {
        'q': location_query,
        'format': 'json',
    }
    response = requests.get(location_url, params=params)
    location_data = response.json()

    if location_data:
        latitude = location_data[0]['lat']
        longitude = location_data[0]['lon']

        # Use latitude and longitude to query weather API
        weather_api_url = f"https://api.weather.gov/points/{latitude},{longitude}"
        time.sleep(1)
        weather_response = requests.get(weather_api_url)
        weather_data = weather_response.json()

        forecast_url = weather_data['properties']['forecast']
        res = requests.get(forecast_url)
        weather_forecast_data = res.json()

        temperature = weather_forecast_data.get('properties', {}).get('periods', [{}])[0].get('temperature', '')
        short_forecast = weather_forecast_data.get('properties', {}).get('periods', [{}])[0].get('shortForecast', '')

        # Store in csv
        with open('events.csv', 'a', newline='') as f:
            writer = csv.writer(f)

            # Write event details and weather information
            writer.writerow([f'Event Name: {names}'])
            writer.writerow([f'Date: {date_location[0]}'])
            writer.writerow([f'Location: {date_location[1]}'])
            writer.writerow([f'Type: {type}'])
            writer.writerow([f'Region: {region}'])
            writer.writerow([f'Temperature: {temperature}ºF'])
            writer.writerow([f'Short Forecast: {short_forecast}'])
            writer.writerow([])